In [ ]:
import json
from pathlib import Path
from whoosh import index

In [ ]:
import os, os.path

if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

ix = index.create_in("indexdir", schema)